## External Tensor Functions

- TVM supports transparent code generation.
- TVM supports black box function calls natively as well. 
- Specifically, TVM supports all tensor functions that are [DLPack](https://github.com/dmlc/dlpack) compatible.


In [1]:
import tvm 
import numpy as np 
from tvm.contrib import cblas

## Use Extern Tensor Function


In [2]:
n = 1024
l = 128
m = 235

In [3]:
bias = tvm.var(name='bias', dtype=tvm.float32)
A = tvm.placeholder(shape=(n, l), name='A')
B = tvm.placeholder(shape=(l, m), name='B')
# Compute several tensor via extern function.
C = tvm.extern(shape=(n, m), inputs=[A, B], 
               fcompute=lambda ins, outs: tvm.call_packed("tvm.contrib.cblas.matmul",
                                                         ins[0], ins[1], outs[0], False, False),
              name="C")

In [4]:
D = tvm.compute(shape=C.shape, fcompute=lambda i, j: C[i, j] + bias, name="D")

In [5]:
s = tvm.create_schedule(D.op)

## Verify the Result

In [6]:
ctx = tvm.cpu(dev_id=0)
f = tvm.build(sch=s, args=[A, B, D, bias], target="llvm")

In [7]:
a = tvm.nd.array(np.random.uniform(size=(n, l)).astype(A.dtype), ctx)
b = tvm.nd.array(np.random.uniform(size=(l, m)).astype(B.dtype), ctx)
d = tvm.nd.array(np.zeros((n, m), dtype=D.dtype), ctx)
bb = 10.0

In [8]:
f(a, b, d, bb)

In [9]:
np.testing.assert_allclose(
    d.asnumpy(), np.dot(a.asnumpy(), b.asnumpy()) + 10, rtol=1e-5)

## Extern Contrib Wrappers

- TVM also provides extern contrib wrappers to useful extern calls, the following line is equivalent to the previous example.

In [10]:
from tvm.contrib import cblas

In [11]:
C = cblas.matmul(lhs=A, rhs=B)
D = tvm.compute(shape=C.shape, fcompute=lambda i, j: C[i, j] + bias, name="D")
s = tvm.create_schedule(D.op)

## Hook Python Function as Extern

- Since we can call into any PackedFunc in TVM. We can use the extern function to callback into python.

- The following example registers a python function into tvm runtime system and use it to complete one stage of the computation. 
- This makes TVM much more flexible. For example, we can insert front-end callbacks to inspect the intermediate results or mix customized code with TVM.

In [12]:
@tvm.register_func("tvm.contrib.my_tvm_addone")
def my_tvm_addone(x, y):
    print("my_tvm_addone signatures: %s, %s" % (type(x), type(y)))
    tvm.nd.array(x.asnumpy() + 1).copyto(y)


In [13]:
A = tvm.placeholder((n,), name="A")
B = tvm.extern(A.shape, inputs=[A], 
               fcompute=lambda ins, outs: tvm.call_packed("tvm.contrib.my_tvm_addone", 
                                              ins[0], outs[0]), 
               name="C")

In [14]:
s = tvm.create_schedule(B.op)
f = tvm.build(s, [A, B], "llvm")
a = tvm.nd.array(np.random.uniform(size=(n,)).astype(A.dtype), ctx)
b = tvm.nd.array(np.random.uniform(size=(n,)).astype(B.dtype), ctx)
f(a, b)
np.testing.assert_allclose(b.asnumpy(), a.asnumpy() + 1, rtol=1e-5)

my_tvm_addone signatures: <class 'tvm.ndarray.NDArray'>, <class 'tvm.ndarray.NDArray'>
